Import IFC

In [8]:
# Import TopologicPy modules
import sys
sys.path.append("C:/Users/sarwj/OneDrive - Cardiff University/Documents/GitHub/topologicpy/src")

In [ ]:
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Graph import Graph
from topologicpy.Helper import Helper

print(Helper.Version())

In [10]:
# ******** CHANGE THIS PATH TO POINT TO YOUR OWN IFC FILE ********
ifc_file_path = r"C:\Users\sarwj\Downloads\IFC2x3_Duplex_Difference_1.ifc"


In [11]:
# ****** CHANGE THIS LIST TO CHOOSE WHAT YOU WANT TO IMPORT FROM YOUR IFC FILE *******
include_types=["IfcSpace", "IfcSlab", "IfcRoof", "IfcWall", "IfcWallStandardCase", "IfcDoor", "IfcWindow"]

In [ ]:
# Create a graph from the IFC path
graph1 = Graph.ByIFCPath(ifc_file_path,
                        includeTypes= include_types,
                        transferDictionaries=True,
                        useInternalVertex=True,
                        storeBREP=True,
                        removeCoplanarFaces=True)

# Extract the topologies from the vertices of the graph
topologies = []
rogue_vertices = [] # These are rogue vertices that have no topology associated with them.
for v in Graph.Vertices(graph1):
    d = Topology.Dictionary(v)
    brep_string = Dictionary.ValueAtKey(d, "brep")
    if brep_string:
        topology = Topology.ByBREPString(brep_string)
        if Topology.IsInstance(topology, "Topology"):
            topology = Topology.SetDictionary(topology, d)
            topologies.append(topology)
        else:
            rogue_vertices.append(v)
    else:
        rogue_vertices.append(v)

# Remove rogue vertices from the graph
for rogue_vertex in rogue_vertices:
    graph1 = Graph.RemoveVertex(graph1, rogue_vertex)

# Give the graph a fake IFC name to be displayed in the legend.
d = Dictionary.ByKeyValue("IFC_name", "Graph")
graph1 = Graph.SetDictionary(graph1, d)
print("Done")

In [13]:
centralities = Graph.ClosenessCentrality(graph1, silent=False)
vertices = Graph.Vertices(graph1)
for v in vertices:
    d = Topology.Dictionary(v)
    c = Dictionary.ValueAtKey(d,"closeness_centrality")
    d = Dictionary.SetValueAtKey(d, "closeness_centrality", c*20+4)
    v = Topology.SetDictionary(v, d)

In [ ]:
# Choose a renderer that works for your environment

# renderer="browser"
# renderer="jupyterlab"
renderer="vscode"

# Draw the topologies and the graph with color-coding
Topology.Show(topologies, graph1,
              nameKey="IFC_name",
              sagitta= 0.05,
              absolute=False,
              faceOpacity=0.1,
              vertexSizeKey="closeness_centrality",
              vertexLabelKey="IFC_name",
              vertexGroupKey="IFC_type",
              vertexGroups=["Unknown", "IfcSpace", "IfcSlab", "IfcRoof", "IfcWall", "IfcWallStandardCase", "IfcDoor", "IfcWindow"],
              showVertexLegend = False,
              showEdgeLegend = False,
              showFaceLegend = False,
              backgroundColor="white",
              width=1024,
              height=900,
              renderer=renderer)